<a href="https://colab.research.google.com/github/ialara/actf/blob/e2e-prototype/absorption_e2e_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
rng = np.random.default_rng()

In [ ]:
rng.binomial(20, 1/20, 10)

array([0, 1, 0, 0, 1, 2, 0, 0, 1, 1])

In [ ]:
np.mean(rng.binomial(144, .05, 20))

6.95

In [ ]:
class Pilot:
    def __init__(self, i, f16_sorties=59, tos=0, experienced=False):
        self.id = i
        self.f16_sorties = f16_sorties
        self.tos = tos
        self.experienced = experienced
        self.arrived_month = 0

    def set_arrived_month(self, month):
        self.arrived_month = month

    def increment_tos(self, months=1):
        self.tos += months

    def increment_f16_sorties(self, sortie_increment=1):
        self.f16_sorties += sortie_increment

    def set_experienced(self, to=True):
        self.experienced = to

    def print_(self):
        print(f'ID: {self.id:2d} | EXP: {"Y" if self.experienced else "N"}  |' \
               f'TOS: {self.tos} | Sorties: {self.f16_sorties}')

class Squadron:
    def __init__(self, name='default_Squadron'):
        self.name = name
        self.pilots = []
        self.pid = 0

    def assign_pilot(self, pilot, arrived_month=0):
        self.pilots.append(pilot)
        pilot.set_arrived_month(arrived_month)

    def set_monthly_sorties_available(self, capacity=240):
        self.monthly_sorties_available = capacity

    def _next_pid(self):
        self.pid += 1
        return self.pid

    def populate_initial(self, num=35):
        for _ in range(num):
            self.assign_pilot(Pilot(self._next_pid()))

    def inflow_pilots(self, num_pilots=15, arrival_month=0):
        for _ in range(num_pilots):
            self.assign_pilot(Pilot(self._next_pid()), arrival_month)

    def outflow_pilots(self, tos_threshold=32):
        removed_pilots = [p for p in self.pilots if p.tos >= tos_threshold]
        self.pilots = [p for p in self.pilots if p not in set(removed_pilots)]
        return removed_pilots

    def fly_month(self, sorties_available=None, INX_sortie_pct=0.6):
        if sorties_available is None:
            sorties_available = self.monthly_sorties_available
        INX_sorties = int(sorties_available * INX_sortie_pct)
        INX_pilots = [p for p in self.pilots if not p.experienced]
        num_INX_pilots = len(INX_pilots)

        rng.shuffle(INX_pilots)
        INX_sorties_remaining = INX_sorties
        for p in INX_pilots:
            if INX_sorties_remaining > 0:
                my_draw = rng.binomial(INX_sorties, 1 / num_INX_pilots)
                my_SCM = min(my_draw, INX_sorties_remaining)
                print(f'PID {p.id:2d}: drew {my_draw:2d}, flying {my_SCM:2d}.')
                p.increment_f16_sorties(my_SCM)
                INX_sorties_remaining -= my_SCM

        print(f'EOM INX sorties remaining: {INX_sorties_remaining}')

    def age_squadron(self, months=1):
        for p in self.pilots:
            p.increment_tos(months)

    def summarize(self):
        for p in self.pilots:
            p.print_()

class Simulation:
    def __init__(self, run_num=0):
        self.run_num = run_num
        self.month_num = 0

    def setup(self, initial_size=35, monthly_sortie_capacity=240):
        self.sq = Squadron('test_Squadron')
        self.sq.populate_initial(initial_size)
        self.sq.set_monthly_sorties_available(monthly_sortie_capacity)

    def step_month(self, num_months=1, inflow_size=10):
        self.month_num += 1
        self.sq.inflow_pilots(inflow_size, self.month_num)
        print('After inflow:')
        self.sq.summarize()
        self.sq.fly_month()
        print('After flying:')
        self.sq.summarize()
        self.sq.age_squadron()
        print('After aging:')
        self.sq.summarize()
        self.sq.outflow_pilots()
        print('After outflow:')
        self.sq.summarize()


In [ ]:
sim = Simulation()
sim.setup(20)
sim.sq.summarize()

ID:  1 | EXP: N  |TOS: 0 | Sorties: 59
ID:  2 | EXP: N  |TOS: 0 | Sorties: 59
ID:  3 | EXP: N  |TOS: 0 | Sorties: 59
ID:  4 | EXP: N  |TOS: 0 | Sorties: 59
ID:  5 | EXP: N  |TOS: 0 | Sorties: 59
ID:  6 | EXP: N  |TOS: 0 | Sorties: 59
ID:  7 | EXP: N  |TOS: 0 | Sorties: 59
ID:  8 | EXP: N  |TOS: 0 | Sorties: 59
ID:  9 | EXP: N  |TOS: 0 | Sorties: 59
ID: 10 | EXP: N  |TOS: 0 | Sorties: 59
ID: 11 | EXP: N  |TOS: 0 | Sorties: 59
ID: 12 | EXP: N  |TOS: 0 | Sorties: 59
ID: 13 | EXP: N  |TOS: 0 | Sorties: 59
ID: 14 | EXP: N  |TOS: 0 | Sorties: 59
ID: 15 | EXP: N  |TOS: 0 | Sorties: 59
ID: 16 | EXP: N  |TOS: 0 | Sorties: 59
ID: 17 | EXP: N  |TOS: 0 | Sorties: 59
ID: 18 | EXP: N  |TOS: 0 | Sorties: 59
ID: 19 | EXP: N  |TOS: 0 | Sorties: 59
ID: 20 | EXP: N  |TOS: 0 | Sorties: 59


In [ ]:
sim.step_month()

After inflow:
ID:  1 | EXP: N  |TOS: 1 | Sorties: 65
ID:  2 | EXP: N  |TOS: 1 | Sorties: 63
ID:  3 | EXP: N  |TOS: 1 | Sorties: 66
ID:  4 | EXP: N  |TOS: 1 | Sorties: 63
ID:  5 | EXP: N  |TOS: 1 | Sorties: 65
ID:  6 | EXP: N  |TOS: 1 | Sorties: 60
ID:  7 | EXP: N  |TOS: 1 | Sorties: 62
ID:  8 | EXP: N  |TOS: 1 | Sorties: 63
ID:  9 | EXP: N  |TOS: 1 | Sorties: 64
ID: 10 | EXP: N  |TOS: 1 | Sorties: 64
ID: 11 | EXP: N  |TOS: 1 | Sorties: 65
ID: 12 | EXP: N  |TOS: 1 | Sorties: 65
ID: 13 | EXP: N  |TOS: 1 | Sorties: 62
ID: 14 | EXP: N  |TOS: 1 | Sorties: 63
ID: 15 | EXP: N  |TOS: 1 | Sorties: 61
ID: 16 | EXP: N  |TOS: 1 | Sorties: 65
ID: 17 | EXP: N  |TOS: 1 | Sorties: 64
ID: 18 | EXP: N  |TOS: 1 | Sorties: 63
ID: 19 | EXP: N  |TOS: 1 | Sorties: 65
ID: 20 | EXP: N  |TOS: 1 | Sorties: 61
ID: 21 | EXP: N  |TOS: 1 | Sorties: 62
ID: 22 | EXP: N  |TOS: 1 | Sorties: 64
ID: 23 | EXP: N  |TOS: 1 | Sorties: 64
ID: 24 | EXP: N  |TOS: 1 | Sorties: 63
ID: 25 | EXP: N  |TOS: 1 | Sorties: 66
ID: 26 | EX